# Find the model

## Global configurations

In [1]:
import logging
import logging.config
import os
import sys
import warnings
from enum import auto
import pandas as pd
import numpy as np
from IPython.core.display import display

rootdir = '/home/hym/trunk/TCRBert'
workdir = '%s/notebook' % rootdir
datadir = '%s/data' % rootdir
srcdir = '%s/tcrbert' % rootdir
outdir = '%s/output' % rootdir

os.chdir(workdir)

sys.path.append(rootdir)
sys.path.append(srcdir)

from tcrbert.exp import Experiment
from tcrbert.predlistener import PredResultRecoder


# Display
pd.set_option('display.max.rows', 2000)
pd.set_option('display.max.columns', 2000)

# Logger
warnings.filterwarnings('ignore')
logging.config.fileConfig('../config/logging.conf')
logger = logging.getLogger('tcrbert')
logger.setLevel(logging.INFO)

# Target experiment
exp_key = 'exp1'
experiment = Experiment.from_key(exp_key)

exp_conf = experiment.exp_conf

display(exp_conf)

2022-11-02 18:26:43 [INFO]: Loaded exp_conf: {'title': 'exp1', 'description': 'Fine-tuning of pre-trained TAPE model in a progressively specialized manner', 'paper': 'exp1', 'model_config': '../config/bert-base/', 'outdir': '../output/exp1', 'train': {'pretrained_model': {'type': 'tape', 'location': '../config/bert-base/'}, 'data_parallel': False, 'backup': 'train.bak.{date}.tar.gz', 'rounds': [{'data': 'dash_vdjdb_mcpas', 'test_size': 0.2, 'batch_size': 128, 'n_epochs': 150, 'n_workers': 12, 'metrics': ['accuracy'], 'optimizer': {'type': 'adam', 'lr': 0.0001}, 'train_bert_encoders': [-10, None], 'early_stopper': {'monitor': 'accuracy', 'patience': 15}, 'model_checkpoint': {'chk': 'train.{round}.model_{epoch}.chk', 'monitor': 'accuracy', 'save_best_only': True, 'period': 1}, 'result': 'train.{round}.result.json'}, {'data': 'iedb_sars2', 'test_size': 0.2, 'batch_size': 128, 'n_epochs': 100, 'n_workers': 12, 'metrics': ['accuracy'], 'optimizer': {'type': 'adam', 'lr': 0.0001}, 'train_ber

{'title': 'exp1',
 'description': 'Fine-tuning of pre-trained TAPE model in a progressively specialized manner',
 'paper': 'exp1',
 'model_config': '../config/bert-base/',
 'outdir': '../output/exp1',
 'train': {'pretrained_model': {'type': 'tape',
   'location': '../config/bert-base/'},
  'data_parallel': False,
  'backup': 'train.bak.{date}.tar.gz',
  'rounds': [{'data': 'dash_vdjdb_mcpas',
    'test_size': 0.2,
    'batch_size': 128,
    'n_epochs': 150,
    'n_workers': 12,
    'metrics': ['accuracy'],
    'optimizer': {'type': 'adam', 'lr': 0.0001},
    'train_bert_encoders': [-10, None],
    'early_stopper': {'monitor': 'accuracy', 'patience': 15},
    'model_checkpoint': {'chk': 'train.{round}.model_{epoch}.chk',
     'monitor': 'accuracy',
     'save_best_only': True,
     'period': 1},
    'result': 'train.{round}.result.json'},
   {'data': 'iedb_sars2',
    'test_size': 0.2,
    'batch_size': 128,
    'n_epochs': 100,
    'n_workers': 12,
    'metrics': ['accuracy'],
    'opt

## Find target aa position in the epitope

In [ ]:
from tcrbert.dataset import TCREpitopeSentenceDataset, CN
from collections import OrderedDict, Counter
from torch.utils.data import DataLoader

epitope = 'YLQPRTFLL'
epitope_len = len(epitope)

sh_ds = TCREpitopeSentenceDataset.from_key('shomuradova')
sh_df = sh_ds.df_enc
im_ds = TCREpitopeSentenceDataset.from_key('immunecode')

# Remove duplicated CDR3beta seqs with Shomuradova
im_ds.df_enc = im_ds.df_enc[
        im_ds.df_enc[CN.cdr3b].map(lambda seq: seq not in sh_df[CN.cdr3b].values)
]
im_df = im_ds.df_enc

n_found = 0
n_train = 0
metrics = ['accuracy', 'f1', 'roc_auc']

target_attn_pos = 3

while(n_found < 5):
    found = True

    logger.info('>>>Begin train, n_train: %s' % n_train)
    
    experiment.train()
    
    logger.info('>>>Done to train, n_train: %s' % n_train)
    n_train = n_train + 1
    
    for i in range(experiment.n_train_rounds):
        train_result = experiment.get_train_result(i)
        logger.info('Round %s train results======================' % i)
        logger.info('n_epochs: %s' % train_result['n_epochs'])
        logger.info('stopped_epoch: %s' % train_result['stopped_epoch'])
        logger.info('best_epoch: %s' % train_result['best_epoch'])
        logger.info('best_score: %s' % train_result['best_score'])
        logger.info('best_chk: %s' % train_result['best_chk'])
    
    
    model = experiment.load_eval_model()
    eval_recoder = PredResultRecoder(output_attentions=True, output_hidden_states=True)
    model.add_pred_listener(eval_recoder)    
    
    for ds, max_cum_ratio in zip([sh_ds, im_ds], [0.9, 0.85]):
        df = ds.df_enc
        data_loader = DataLoader(ds, batch_size=len(ds), shuffle=False, num_workers=2)
        logger.info('Predicting for %s' % ds.name)
        model.predict(data_loader=data_loader, metrics=metrics)
        logger.info('Performace score_map for %s: %s' % (ds.name, eval_recoder.result_map['score_map']))
        
        output_labels = np.array(eval_recoder.result_map['output_labels'])
        
        # Select target CDR3b sequences with most common lengths
        pos_indices = np.where(output_labels == 1)[0]
        # print('pos_indices: %s(%s)' % (pos_indices, str(pos_indices.shape)))
        pos_cdr3b = df[CN.cdr3b].values[pos_indices]

        lens, cnts = zip(*sorted(Counter(map(lambda x: len(x), pos_cdr3b)).items()))
        lens = np.array(lens)
        cnts = np.array(cnts)

        # Select target indices by cdr3b sequence lenghts
        target_index_map = OrderedDict()
        order = np.argsort(cnts)[::-1]
        cum_cnt = 0
        for cur_len, cur_cnt in zip(lens[order], cnts[order]):
            cum_cnt += cur_cnt
            cum_ratio = cum_cnt/pos_indices.shape[0]
            if cum_ratio < max_cum_ratio:
                target_indices = np.where((output_labels == 1) & (df[CN.cdr3b].map(lambda x: len(x) == cur_len)))[0]
                logger.debug('target_indices for %s: %s(%s)' % (cur_len, target_indices, target_indices.shape[0]))
                target_index_map[cur_len] = target_indices
        
        # Investigate attention weights
        attentions = eval_recoder.result_map['attentions']
        # attentions.shape: (n_layers, n_data, n_heads, max_len, max_len)
        logger.info('attentions.shape: %s' % str(attentions.shape))

        for i, (cur_len, cur_indices) in enumerate(target_index_map.items()):
            attns = attentions[:, cur_indices]
            sent_len = epitope_len + cur_len

            # Marginalized position-wise attentions by mean
            attns = np.mean(attns, axis=(0, 1, 2, 3))[1:sent_len+1]
            logger.info('Marginalized attns for cdr3b %s: %s (%s)' % (cur_len, attns, str(attns.shape)))
            
            epi_attns = attns[:epitope_len]
            cur_max_attn_pos = np.argmax(epi_attns)
            logger.info('Current max epitope attention weight: %s at %s' % (epi_attns[cur_max_attn_pos], 
                                                                            cur_max_attn_pos))
            if target_attn_pos != cur_max_attn_pos:
                found = False
 
    if found:
        logger.info('>>>>>Found it!, backup train results, n_found: %s' % n_found)
        experiment.backup_train_results()
        n_found = n_found + 1
            

2022-11-02 18:27:00 [INFO]: shomuradova dataset was loaded from ../output/shomuradova.data.csv, df.shape: (610, 9)
2022-11-02 18:27:00 [INFO]: immunecode dataset was loaded from ../output/immunecode.data.csv, df.shape: (742, 9)
2022-11-02 18:27:00 [INFO]: >>>Begin train 0
2022-11-02 18:27:00 [INFO]: ======================
2022-11-02 18:27:00 [INFO]: Begin train at 2022-11-02 18:27:00.161968
2022-11-02 18:27:00 [INFO]: Loading the TAPE pretrained model from ../config/bert-base/
2022-11-02 18:27:02 [INFO]: Start 2 train rounds of exp1 at 2022-11-02 18:27:00.161968
2022-11-02 18:27:02 [INFO]: train_conf: {'pretrained_model': {'type': 'tape', 'location': '../config/bert-base/'}, 'data_parallel': False, 'backup': 'train.bak.{date}.tar.gz', 'rounds': [{'data': 'dash_vdjdb_mcpas', 'test_size': 0.2, 'batch_size': 128, 'n_epochs': 150, 'n_workers': 12, 'metrics': ['accuracy'], 'optimizer': {'type': 'adam', 'lr': 0.0001}, 'train_bert_encoders': [-10, None], 'early_stopper': {'monitor': 'accuracy

Validating in epoch 0/150: 100%|██████████| 40/40 [00:36<00:00,  1.11batch/s]

2022-11-02 18:30:47 [INFO]: [EvalScoreRecoder]: In epoch 0/150, loss train score: 0.6765513669086408, val score: 0.6619814932346344
2022-11-02 18:30:47 [INFO]: [EvalScoreRecoder]: In epoch 0/150, accuracy train score: 0.5666676084990958, val score: 0.604709201388889
2022-11-02 18:30:47 [INFO]: [EarlyStopper]: In epoch 0/150, accuracy score: 0.604709201388889, best accuracy score: -inf;update best score to 0.604709201388889
2022-11-02 18:30:47 [INFO]: [ModelCheckpoint]: Checkpoint at epoch 0: accuracy improved from -inf to 0.604709201388889, saving model to ../output/exp1/train.0.model_0.chk



Validating in epoch 1/150: 100%|██████████| 40/40 [00:35<00:00,  1.12batch/s]

2022-11-02 18:34:26 [INFO]: [EvalScoreRecoder]: In epoch 1/150, loss train score: 0.5937079911745047, val score: 0.6040216721594334
2022-11-02 18:34:26 [INFO]: [EvalScoreRecoder]: In epoch 1/150, accuracy train score: 0.6544204905063291, val score: 0.6717447916666666
2022-11-02 18:34:26 [INFO]: [EarlyStopper]: In epoch 1/150, accuracy score: 0.6717447916666666, best accuracy score: 0.604709201388889;update best score to 0.6717447916666666
2022-11-02 18:34:26 [INFO]: [ModelCheckpoint]: Checkpoint at epoch 1: accuracy improved from 0.604709201388889 to 0.6717447916666666, saving model to ../output/exp1/train.0.model_1.chk


...


Validating in epoch 3/100: 100%|██████████| 155/155 [02:18<00:00,  1.12batch/s]

2022-11-04 13:57:45 [INFO]: [EvalScoreRecoder]: In epoch 3/100, loss train score: 0.6935147407375626, val score: 0.693393741115447
2022-11-04 13:57:45 [INFO]: [EvalScoreRecoder]: In epoch 3/100, accuracy train score: 0.501544773095624, val score: 0.5031754032258065
2022-11-04 13:57:45 [INFO]: [EarlyStopper]: In epoch 3/100, accuracy score: 0.5031754032258065, best accuracy score: 0.5031754032258065;accuracy score was not improved
2022-11-04 13:57:45 [INFO]: [EarlyStopper]: Current wait count: 3, patience: 10
2022-11-04 13:57:45 [INFO]: [ModelCheckpoint]: Checkpoint at epoch 3: accuracy did not improve



Validating in epoch 4/100: 100%|██████████| 155/155 [02:11<00:00,  1.18batch/s]

2022-11-04 14:10:27 [INFO]: [EvalScoreRecoder]: In epoch 4/100, loss train score: 0.6933955370510416, val score: 0.6931733796673436
2022-11-04 14:10:27 [INFO]: [EvalScoreRecoder]: In epoch 4/100, accuracy train score: 0.500430510534846, val score: 0.49682459677419355
2022-11-04 14:10:27 [INFO]: [EarlyStopper]: In epoch 4/100, accuracy score: 0.49682459677419355, best accuracy score: 0.5031754032258065;accuracy score was not improved
2022-11-04 14:10:27 [INFO]: [EarlyStopper]: Current wait count: 4, patience: 10
2022-11-04 14:10:27 [INFO]: [ModelCheckpoint]: Checkpoint at epoch 4: accuracy did not improve



Training in epoch 5/100:  37%|███▋      | 227/617 [03:51<06:34,  1.01s/batch]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Validating in epoch 8/100:  50%|████▉     | 77/155 [01:06<01:06,  1.17batch/s]